In [27]:
from collections import Counter
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np


In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('../reviews_Books_5.json.gz')


In [ ]:
df.shape

In [10]:
import pickle
df.to_pickle('reviews_Books_5.h5')

In [11]:
print df[:2]

       reviewerID        asin     reviewerName helpful  unixReviewTime  \
0    AIXZKN4ACSKI  1881509818     David Briner  [0, 0]      1390694400   
1  A1L5P841VIO02V  1881509818  Jason A. Kramer  [1, 1]      1328140800   

                                          reviewText  overall   reviewTime  \
0  This came in on time and I am veru happy with ...      5.0  01 26, 2014   
1  I had a factory Glock tool that I was using fo...      5.0   02 2, 2012   

                             summary  
0                     Woks very good  
1  Works as well as the factory tool  


In [128]:
from sklearn.utils import shuffle
df = pd.read_pickle('reviews_Books_5.h5')
df.to_csv('reviews_Books_5.csv')
df = shuffle(df)

In [129]:
from sklearn.model_selection import train_test_split

Training_DataFrame, Testing_DataFrame = train_test_split(df, test_size=0.2)

In [130]:
print Training_DataFrame.shape
print Testing_DataFrame.shape

(237069, 9)
(59268, 9)


In [131]:
print "Training dataset details:"
Uniq_movies_Train =  len(set(Training_DataFrame['asin'].values))
print "Number of item:", Uniq_movies_Train
Uniq_users_Train =   len(set(Training_DataFrame['reviewerID'].values))
print "Number of Users:", Uniq_users_Train
Uniq_ratings_Train = len(set(Training_DataFrame['overall'].values))
print "Number of ratings:", Uniq_ratings_Train
print "\n"

Training dataset details:
Number of item: 18356
Number of Users: 35589
Number of ratings: 5




In [132]:
#Training_DataFrame=Training_DataFrame[:5000]
Training_DataFrame.columns
# Lists of unique movies and users
movie_list = list(set(Training_DataFrame['asin'].values))
user_list = list(set(Training_DataFrame['reviewerID'].values))

print Training_DataFrame[0:2]
#Storing the indices of users to generate the Rating Memory Matrix
UserIndices={}
for i in range(len(user_list)):
    UserIndices[user_list[i]] = i

#Storing the indices of movies to generate the Rating Memory Matrix
#These indices will be used to populate the UsersRatingMemory for Memory based Collaborative Filtering
MovieIndices={}
for j in range(len(movie_list)):
    MovieIndices[movie_list[j]] = j
    
#UsersRatingMemory stores the users and corresponding movies ratings in Matrix form
UsersRatingMemory=np.zeros((Uniq_users_Train,Uniq_movies_Train))

for i in range(0, len(Training_DataFrame)):
    movie_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and movie indices
    user_ind=UserIndices[user_id]
    movie_ind=MovieIndices[movie_id]
    rating=Training_DataFrame.iloc[i]['overall']
    #populate the matrix with rating at corresponding user and movie index
    UsersRatingMemory[user_ind][movie_ind]=float(rating)

AvgRatings = np.true_divide(UsersRatingMemory.sum(1),(UsersRatingMemory!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRatings

            reviewerID        asin reviewerName helpful  unixReviewTime  \
209248   A99N6WDOKAFFA  B0049ZFP7U     Goldencr  [1, 1]      1360195200   
181919  A2D0CO1OA6DSWY  B0038ZYRGC          Gil  [0, 0]      1360195200   

                                               reviewText  overall  \
209248  The description says for 26 but it works great...      4.0   
181919  I have these on for hours, they are comfortabl...      4.0   

        reviewTime            summary  
209248  02 7, 2013  Great Fit on 29er  
181919  02 7, 2013      Why pay more?  
Average Ratings of the Users are:
[4.16666667 4.25       4.8        ... 4.11764706 4.8        2.33333333]


In [136]:
from scipy.stats.stats import pearsonr

def PredictUserRating(user_id, movie_id):
    '''This function predicts the rating based on user_id and movie_id'''
    movie_ind = MovieIndices[movie_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the movie
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory[:,movie_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #MovieRatingsOthers: stores other users ratings
    MovieRatingsOthers = UsersRatingMemory[ratedindices, movie_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory[ind],UsersRatingMemory[user_ind])[0])
    k=0.001
    # Calculate the prediction
    pred = AvgRatings[user_ind] + k * np.sum( PearsonCoeffs* (MovieRatingsOthers - AvgRatings[ratedindices]))
    return pred

def Test(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        movie_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRating(user_id,movie_id)
    return Predictions

In [140]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print Testing_DataFrame.columns
Predictions=Test(1000)
print "For 10 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'].values[:1000])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:1000]))

Index([u'reviewerID', u'asin', u'reviewerName', u'helpful', u'unixReviewTime',
       u'reviewText', u'overall', u'reviewTime', u'summary'],
      dtype='object')
For 10 Samples:
The Mean Absolute Error is: 0.6741605049396879
Root Mean Square Error is: 0.9926953965814299


In [147]:
rating = [mem for mem in UsersRatingMemory[:][1] if mem ]
 

In [148]:
print rating

[5.0, 3.0, 5.0, 4.0]


In [169]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()
UsersRatingMemory_sentiment=np.zeros((Uniq_users_Train,Uniq_movies_Train))

for i in range(0, len(Training_DataFrame)):
    movie_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and movie indices
    user_ind=UserIndices[user_id]
    movie_ind=MovieIndices[movie_id]
    rating=float(Training_DataFrame.iloc[i]['overall'])
    
    if Training_DataFrame.iloc[i]['reviewText']:
    #populate the matrix with rating at corresponding user and movie index
        sentiment_rating=sent.polarity_scores(Training_DataFrame.iloc[i]['reviewText'])['compound']
        rating=sentiment_rating+rating
    
    UsersRatingMemory_sentiment[user_ind][movie_ind]=rating

AvgRatings = np.true_divide(UsersRatingMemory_sentiment.sum(1),(UsersRatingMemory_sentiment!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRatings

Average Ratings of the Users are:
[4.68426667 4.33795    5.39975    ... 4.97775882 5.5613     2.6225    ]


In [150]:
def PredictUserRatingsentiment(user_id, movie_id):
    '''This function predicts the rating based on user_id and movie_id'''
    movie_ind = MovieIndices[movie_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the movie
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory_sentiment[:,movie_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #MovieRatingsOthers: stores other users ratings
    MovieRatingsOthers = UsersRatingMemory_sentiment[ratedindices, movie_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory_sentiment[ind],UsersRatingMemory_sentiment[user_ind])[0])
    k=0.001
    # Calculate the prediction
    pred = AvgRatings[user_ind] + k * np.sum( PearsonCoeffs* (MovieRatingsOthers - AvgRatings[ratedindices]))
    return pred

def Test(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        movie_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRatingsentiment(user_id,movie_id)
    return Predictions

In [170]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print Testing_DataFrame.columns
Predictions=Test(1000)
print "For 10 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'][:1000])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:1000]))

Index([u'reviewerID', u'asin', u'reviewerName', u'helpful', u'unixReviewTime',
       u'reviewText', u'overall', u'reviewTime', u'summary'],
      dtype='object')
For 10 Samples:
The Mean Absolute Error is: 0.8428040351856583
Root Mean Square Error is: 1.190641797190995


In [1]:
m = [mem for mem in UsersRatingMemory_sentiment[:][1] if mem ]
print m
m = [mem for mem in UsersRatingMemory[:][1] if mem ]
print m

NameError: name 'UsersRatingMemory_sentiment' is not defined

[0. 0. 0. ... 0. 0. 0.]
